In [ ]:
import IPython.display
import numpy as np
import PIL.Image
import scipy.stats
import seaborn as sns
from matplotlib import cm
from ipywidgets import interact
from ipywidgets import IntSlider
from matplotlib import patches
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import AutoMinorLocator
from io import BytesIO

In [ ]:
# Source: https://stackoverflow.com/questions/8056458/display-image-with-a-zoom-1-with-matplotlib-imshow-how-to

def showbytes(a):
    IPython.display.display(IPython.display.Image(data=a))

def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [ ]:
def get_random_walk_transition_matrix(n, p=0.5):
    """Get transition matrix for markov chain with states corresponding to
    values in the range ``[-n, n]`` (``2*n+1`` possible states), with
    each state transition having probability ``p`` of going up and
    probability ``1-p`` of going down.
    """
    m = n * 2 + 1
    mat = p * np.eye(m, m, k=1) + (1 - p) * np.eye(m, m, k=-1)
    mat[0, 0] = 1 - p
    mat[-1, -1] = 1 - p
    return mat

get_random_walk_transition_matrix(3, 0.6)

In [ ]:
@interact(
    n=(1, 200),
    max_t=(1, 500),
    p=(0., 1., 0.01),
)
def f(n=5, max_t=10, p=0.5):
    transition_matrix = get_random_walk_transition_matrix(n, p)
    initial_state = np.array([0] * n + [1] + [0] * n)
    states = [np.linalg.matrix_power(transition_matrix, t).dot(initial_state) for t in range(max_t)]
    im = np.stack([state / state.max() for state in states]).T * 255
    plt.imshow(im, cmap='gray')

In [ ]:
def get_poissonlike_transition_matrix(n, p=0.5):
    """Get transition matrix for markov chain with states corresponding to
    values in the range ``[0, n]`` (``n+1`` possible states), with
    each state transition having probability ``p`` of going up and
    probability ``1-p`` of staying put.
    """
    m = n + 1
    return p * np.eye(m, m, k=1) + (1 - p) * np.eye(m, m)


get_poissonlike_transition_matrix(5, 0.4)

In [ ]:
max_max_y = 200
max_max_t = 500

@interact(
    max_t=(1, max_max_t),
    max_y=(1, max_max_y),
    p=(0., 1., 0.001),
    t=(-1, max_max_t),
    y=(-1, max_max_y),
)
def f(max_t=24, max_y=14, p=0.3, t=10, y=2):
    transition_matrix = get_poissonlike_transition_matrix(max_y, p)
    initial_state = np.array([1] + [0] * max_y)
    t_values = np.arange(max_t + 1)
    y_values = np.arange(max_y + 1)
    states = [np.linalg.matrix_power(transition_matrix, t).transpose().dot(initial_state) for t in t_values]
    state_grid = np.stack(states).T
    state_grid_norm = np.stack([state / state.max() for state in states]).T
    
    fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(
        2,
        2,
        figsize=(14, 10),
        gridspec_kw={
            'width_ratios': [1.618, 1.],
            'height_ratios': [1.618, 1.],
        },
        constrained_layout=True,
    )
    t_grid, y_grid = np.meshgrid(t_values, y_values)
    ax0.pcolormesh(
        t_grid,
        y_grid,
        1 - state_grid_norm * 0.6,
        cmap='gray',
        vmin=0,
        vmax=1,
    )
    if 0 <= y < max_y:
        if max_y < 50:
            ax0.axhline(y, color='limegreen', lw=2, alpha=0.5)
            ax0.axhline(y + 1, color='limegreen', lw=2, alpha=0.5)
        ax0.add_patch(patches.Rectangle((0, y), max_t, 1, facecolor='limegreen', alpha=0.4))
    if 0 <= t < max_t:
        ax0.axvline(t, color='dodgerblue', lw=2, alpha=0.5)
        ax0.axvline(t + 1, color='dodgerblue', lw=2, alpha=0.5)
        ax0.add_patch(patches.Rectangle((t, 0), 1, max_y, facecolor='dodgerblue', alpha=0.4))

    ax0.set_xlabel('Time')
    ax0.set_ylabel('Y value')

    if max_t < 30:
        ax0.xaxis.set_major_locator(MultipleLocator(1))
        ax0.xaxis.set_minor_locator(AutoMinorLocator(1))
    ax0.grid(True, axis='x')
    if max_y < 25:
        ax0.yaxis.set_major_locator(MultipleLocator(1))
        ax0.yaxis.set_minor_locator(AutoMinorLocator(1))
    ax0.grid(True, axis='y')

    if 0 <= t < max_t:
        y_values2 = np.arange(0, max_y + 1, 0.1)
        ty_values = scipy.stats.norm.pdf(y_values2, loc=p * t, scale=np.sqrt(t) / 2)
        
        ax1.barh(y_values, state_grid[:, t], color='dodgerblue', alpha=0.5)
        ax1.plot(ty_values, y_values2, 'k--', alpha=0.5)
        ax1.legend(['Normal', 'Binomial'])
        ax1.set_title('P(Y) at particular time as function of y')
        ax1.grid(axis='y')
        ax1.set_xlabel('Probability')
        ax1.set_ylabel('Y value')
        if max_y < 30:
            ax1.yaxis.set_major_locator(MultipleLocator(1))
            ax1.yaxis.set_minor_locator(AutoMinorLocator(1))

    if 0 <= y < max_y:
        t_values2 = np.arange(0, max_t + 1, 0.1)
        yt_values = scipy.stats.poisson.pmf(y, p * t_values2)

        ax2.bar(t_values, state_grid[y, :], color='limegreen', alpha=0.5)
        ax2.plot(t_values2, yt_values, 'k--', alpha=0.5)
        ax2.legend(['Poisson', 'Negative binomial'])
        ax2.set_title('P(Y) for particular Y as a function of t')
        ax2.set_xlim(0, max_t)
        ax2.grid(axis='x')
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Probability')
        if max_t < 30:
            ax2.xaxis.set_major_locator(MultipleLocator(1))
            ax2.xaxis.set_minor_locator(AutoMinorLocator(1))

    ax3.axis('off')

In [ ]:
@interact(
    p=(0., 1., 0.01),
    max_t=(1, 1000, 1),
    max_y=(1, 20, 1),
)
def f(
    p=0.1,
    max_t=50,
    max_y=5,
):
    t_values = np.linspace(0, max_t, 100)
    y_values = np.arange(max_y)
    t_grid, y_grid = np.meshgrid(t_values, y_values)
    pmf_grid = scipy.stats.poisson.pmf(y_grid, p * t_grid)
    pmf_grid = np.vstack((pmf_grid, np.ones(t_values.shape)))
    labels = list(y_values) + [f'>={max_y}']

    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    ax.set_title('Stacked Poisson distribution')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    ax.set_xlim(0, max_t)
    ax.set_ylim(0, 1)
    ax.stackplot(t_values, pmf_grid, labels=labels, alpha=0.4)
    ax.legend()

In [ ]:
max_max_y = 200
max_max_t = 500

def get_special_transition_matrix(n, p=0.2, q=0.2):
    """Markov chain transition matrix with `p` probability of going up, `q`
    probability of going down, and `1-p-q` probability of staying still.
    """
    m = n + 1
    return p * np.eye(m, m, k=1) + q * np.eye(m, m, k=-1) + (1 - p - q) * np.eye(m, m)


@interact(
    max_t=(1, max_max_t),
    max_y=(1, max_max_y),
    p=(0., 1., 0.001),
    q=(0., 1., 0.001),
    t=(-1, max_max_t),
    y=(-1, max_max_y),
)
def f(max_t=24, max_y=14, p=0.3, q=0.3, t=10, y=2):
    if p + q > 1.:
        q = 1 - p
    transition_matrix = get_special_transition_matrix(max_y, p, q)
    initial_state = np.array([1] + [0] * max_y)
    t_values = np.arange(max_t + 1)
    y_values = np.arange(max_y + 1)
    states = [np.linalg.matrix_power(transition_matrix, t).transpose().dot(initial_state) for t in t_values]
    state_grid = np.stack(states).T
    state_grid_norm = np.stack([state / state.max() for state in states]).T
    
    fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(
        2,
        2,
        figsize=(14, 10),
        gridspec_kw={
            'width_ratios': [1.618, 1.],
            'height_ratios': [1.618, 1.],
        },
        constrained_layout=True,
    )
    t_grid, y_grid = np.meshgrid(t_values, y_values)
    ax0.pcolormesh(
        t_grid,
        y_grid,
        1 - state_grid_norm * 0.6,
        cmap='gray',
        vmin=0,
        vmax=1,
    )
    if 0 <= y < max_y:
        if max_y < 50:
            ax0.axhline(y, color='limegreen', lw=2, alpha=0.5)
            ax0.axhline(y + 1, color='limegreen', lw=2, alpha=0.5)
        ax0.add_patch(patches.Rectangle((0, y), max_t, 1, facecolor='limegreen', alpha=0.4))
    if 0 <= t < max_t:
        ax0.axvline(t, color='dodgerblue', lw=2, alpha=0.5)
        ax0.axvline(t + 1, color='dodgerblue', lw=2, alpha=0.5)
        ax0.add_patch(patches.Rectangle((t, 0), 1, max_y, facecolor='dodgerblue', alpha=0.4))

    ax0.set_xlabel('Time')
    ax0.set_ylabel('Y value')

    if max_t < 30:
        ax0.xaxis.set_major_locator(MultipleLocator(1))
        ax0.xaxis.set_minor_locator(AutoMinorLocator(1))
    ax0.grid(True, axis='x')
    if max_y < 25:
        ax0.yaxis.set_major_locator(MultipleLocator(1))
        ax0.yaxis.set_minor_locator(AutoMinorLocator(1))
    ax0.grid(True, axis='y')

    if 0 <= t < max_t:
        y_values2 = np.arange(0, max_y + 1, 0.1)
        ty_values = scipy.stats.norm.pdf(y_values2, loc=p * t, scale=np.sqrt(t) / 2)
        
        ax1.barh(y_values, state_grid[:, t], color='dodgerblue', alpha=0.5)
        ax1.plot(ty_values, y_values2, 'k--', alpha=0.5)
        ax1.legend(['Normal', 'Binomial'])
        ax1.set_title('P(Y) at particular time as function of y')
        ax1.grid(axis='y')
        ax1.set_xlabel('Probability')
        ax1.set_ylabel('Y value')
        if max_y < 30:
            ax1.yaxis.set_major_locator(MultipleLocator(1))
            ax1.yaxis.set_minor_locator(AutoMinorLocator(1))

    if 0 <= y < max_y:
        t_values2 = np.arange(0, max_t + 1, 0.1)
        yt_values = scipy.stats.poisson.pmf(y, p * t_values2)

        ax2.bar(t_values, state_grid[y, :], color='limegreen', alpha=0.5)
        ax2.plot(t_values2, yt_values, 'k--', alpha=0.5)
        ax2.legend(['Poisson', 'Negative binomial'])
        ax2.set_title('P(Y) for particular Y as a function of t')
        ax2.set_xlim(0, max_t)
        ax2.grid(axis='x')
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Probability')
        if max_t < 30:
            ax2.xaxis.set_major_locator(MultipleLocator(1))
            ax2.xaxis.set_minor_locator(AutoMinorLocator(1))

    ax3.axis('off')